# Deep Learning : Forward/backward and Gradient Descent

In this notebook I will implement:
Simple forward and backward pass on a custom made numpy ANN based on my the derivations in the pdf [here](https://github.com/YoelGraumann/DeepLearning-Simple-Implementation/blob/main/Forward%20and%20Backward%20pass%20ANN.pdf)

And the gradient descent algorithm which includes
Vanilla GD, SGD, GD with momentum, GD with exponential decay

### Forward and backward pass implementation, based my derivations in pdf i posted on github

Each fully connected layer object must be initialized with:
* neurons_before : the number of neurons or inputs from previous layer. -integer
* current_neurons: the number of neurons in this layer. - integer
* weights: =1 initializes the weights_matrix to be a matrix of 1s, by default they will be initialized by the standard normal distribution
* bias: =1 initializes the bias_vector to be a vector of 1s, =0 initializes the bias vector to be of 0s,by default they will be initialized by the standard normal distribution.

Note that the bias_vecor will a row vector of 1 by the number of the current neurons
note that the weights_matrix is a matrix where the columns correspond to the neurons in the current layer. In other words:
the weights in column 1, will be the weights used for the input of neuron 1 in the hidden layer, and so on.

the forward_step will do a forward step through the fully connected layer, saving x for layer 
the forward_step also calculates z=X*W+B

the backward_step will do a backward step through the layer. It will calculate the grad w.r.t
weights biases and the inputs

In [41]:
import numpy as np
import random
class Fully_Connected_Layer:  
    ## initialization:
    ### the weights=10 and bias=10 will ensure that if there is no input, they will be initialized by the standard normal dist
    def __init__(self,neurons_before,current_neurons,weights=10,bias=10):
        ## default initializations
        self.weights_matrix=np.random.standard_normal((neurons_before,current_neurons))
        self.bias_vector=np.random.standard_normal((1,current_neurons))
        # scanning for wanted weights and biases
        'Note that it does not make any sense to initialize the' 
        'weights_matrix to be of 0s since it will kill the Neural Network'
        if weights==1:
            self.weights_matrix=np.ones((neurons_before,current_neurons))
        if bias==1:
            self.bias_vector=np.ones((1,current_neurons))
        if bias==0:
            self.bias_vector=np.zeros((1,current_neurons))                                   
    ## forward pass                                   
    def forward_step(self,x):
        ## Saving the original input values for the forward step for later in self.x
        self.x=x
        ### calculating z which is , which is X*W+B (no need to transpose the weights_matrix since its created accordingly)
        self.z=np.dot(x,self.weights_matrix)+self.bias_vector
        
        
    def backward_step(self,grad_vals,use_matrix_form=0):
        ## gradients on parameters
        '''
        Note that use_matrix_form is for the edge case when backpropagating to the first layer of the NN
        gives a bunch of shape errors. So when doing a backward step for the first layer of the NN
        do: use_matrix_form=1
        '''
        if use_matrix_form==0:
            self.grad_inputs=np.dot(grad_vals,self.weights_matrix.T)
            self.grad_biases=np.sum(grad_vals,axis=0,keepdims=True)
            self.grad_weights=np.dot(self.x.T,grad_vals)
        else:
            self.grad_inputs=np.dot(grad_vals,self.weights_matrix.T)
            self.grad_biases=np.sum(grad_vals,axis=0,keepdims=True)
            self.grad_weights=np.matrix(self.x).T*np.matrix(grad_vals)
            

Note: The derivative of the sigmoid function is :
$$\sigma(x)(1-\sigma(x))$$
I'll use that fact for the backward_step

Now I will create classes for Relu, sigmoid and identity.
Their forward pass will take in an input x  and save them for later, then output the y
Their backward pass will calculate the derivatives and will be able to send them to the next layer in the backprop algo.
All the Activation Layers will be in the following code block:


In [42]:
## Identity activation layer which was used to understand the layers, worth keeping in.

class Identity_Activation_Layer:
    def forward_step(self,x):
        self.x=x
        self.y=x
    def backward_step(self,grad_vals):
        self.grad_inputs=grad_vals.copy()
        
### ReLU activation Layer
class ReLU_Activation_Layer:
    '''
    ReLU's output is the maximum of the (0,x)
    it will backprop the gradvalus from before if they are non-zero positive,
    else it will backprop 0.
    '''
    ### forward step definition
    def forward_step(self,x):
        ## the input is x
        self.x=x
        ### The output is y.
        self.y=np.maximum(0,x)
    ### backwardward step:
    def backward_step(self,grad_vals):
        #saving to grad_inputs so as to not change the original grad_vals input coming in during backprop
        self.grad_inputs=grad_vals.copy()
        self.grad_inputs[self.grad_inputs<=0]=0
       
    
### Sigmoid activation layer
class Sigmoid_Activation_Layer:
    ## forwardstep
    def forward_step(self,x):
        ## as always, saving the x's for later.
        self.x=x
        #y is the output, which is equal to the sigmoid.
        self.y=1/(1+np.exp(-x))
    def backward_step(self,grad_vals):
        ### derivative of the sigmoid, and by applying the chain rule we use the prev grad_vals
        self.grad_inputs=self.y*(1-self.y)*grad_vals
       
        

The final ingerdient in our code is to create the RSS loss function. This function will calculate the loss on its forward_step, and calculate the derivative on its backward_step, starting the chain of events of back propagation.
Remember that the RSS is given by:


$$RSS=\sum_{i=1}^n (Y_i-\hat{Y_i})^2$$

Where $$\hat{Y_i}$$ is the is the final hidden layer's prediction, and $$Y_i$$ is the real value

Hence I will use this formula for backpropagation:

$$-2(Y_i-\hat{Y_i})$$

In [43]:
class RSS_Loss:
    #simply calculating the loss.
    def forward_step(self,y_real,y_hat):
        #exactly as we defined above
        self.rss_loss=np.sum((y_real-y_hat)**2,axis=-1)
    # backproping
    def backward_step(self,y_real,y_hat):
        #exactly as we defined above.
        self.grad_vals=-2*(y_real-y_hat)

##### Using the above classes / objects, I will create the ANN network I derived in the pdf.

###### Creating the model:
The model has an input of X=[1,2,-1] and an output of Y=0.
Two hidden layers, where both contain 2 neurons each. The activation of these hidden layers is ReLU.
the output layer with a single neuron with the identity as it's activation.
All weights of the networks are initialized to 1.
* Note that I assumed a bias of 0 for all neurons when doing the backprop calculations in the pdf. Will assume the same here.

In [44]:
#input
X=[1,2,-1]
Y=0

###### Declaring the Neural Network

In [45]:
## the first layer gets 3 inputs, it has 2 neurons, the weights are all 1 and the biases are all 0
Hidden_Layer_1=Fully_Connected_Layer(neurons_before=3,current_neurons=2,weights=1,bias=0)
## hidden layer 1 goes through a relu activation.

Activation_of_Layer_1=ReLU_Activation_Layer()

# look at the drawing in the pdf to understand further. same logic as before though:
Hidden_Layer_2=Fully_Connected_Layer(neurons_before=2,current_neurons=2,weights=1,bias=0)

Activation_of_Layer_2=ReLU_Activation_Layer()

### in question 1 we called the final output layer H 3, so we will do the same here
Hidden_Layer_3=Fully_Connected_Layer(neurons_before=2,current_neurons=1,weights=1,bias=0)


# The hidden layer 3 has the identity as the activation...

Activation_of_Layer_3=Identity_Activation_Layer()

## We learned during Lecture 1 that it is useful to see the loss function as a layer. we will do that here:

Loss_Layer=RSS_Loss()

###### FEED FORWARD!
Printing the output for every step so you can see we got the same values both on paper and in code:

In [46]:
Hidden_Layer_1.forward_step(X)
print(Hidden_Layer_1.z)

[[2. 2.]]


In [47]:
Activation_of_Layer_1.forward_step(Hidden_Layer_1.z)
print(Activation_of_Layer_1.y)

[[2. 2.]]


In [48]:
Hidden_Layer_2.forward_step(Activation_of_Layer_1.y)
print(Hidden_Layer_2.z)

[[4. 4.]]


In [15]:
Activation_of_Layer_2.forward_step(Hidden_Layer_2.z)
print(Activation_of_Layer_2.y)

[[4. 4.]]


In [16]:
Hidden_Layer_3.forward_step(Activation_of_Layer_2.y)
print(Hidden_Layer_3.z)

[[8.]]


In [17]:
Activation_of_Layer_3.forward_step(Hidden_Layer_3.z)
print(Activation_of_Layer_3.y)

[[8.]]


In [18]:
Loss_Layer.forward_step(Y,Activation_of_Layer_3.y)
print(Loss_Layer.rss_loss)

[64.]


We can see that our Forward Pass code yields the same numbers as we have calculated in Question 1.

###### BACKPROPAGATION!

In [19]:
Loss_Layer.backward_step(Y,Activation_of_Layer_3.y)
print('The derivative of the loss w.r.t H3 is ',Loss_Layer.grad_vals)

The derivative of the loss w.r.t H3 is  [[16.]]


In [20]:
Activation_of_Layer_3.backward_step(Loss_Layer.grad_vals)
print(Activation_of_Layer_3.grad_inputs)

[[16.]]


In [21]:
Hidden_Layer_3.backward_step(Activation_of_Layer_3.grad_inputs)
print(Hidden_Layer_3.grad_inputs)

[[16. 16.]]


In [22]:
Activation_of_Layer_2.backward_step(Hidden_Layer_3.grad_inputs)
print(Activation_of_Layer_2.grad_inputs)

[[16. 16.]]


In [23]:
Hidden_Layer_2.backward_step(Activation_of_Layer_2.grad_inputs)
print(Hidden_Layer_2.grad_inputs)

[[32. 32.]]


In [24]:
Activation_of_Layer_1.backward_step(Hidden_Layer_2.grad_inputs)
print(Activation_of_Layer_1.grad_inputs)

[[32. 32.]]


In [25]:
Hidden_Layer_1.backward_step(Activation_of_Layer_1.grad_inputs,use_matrix_form=1)
## Done backpropagation.

Let's print the derivatives w.r.t. weights and biases  for $$H^{(3)}$$

In [26]:
Hidden_Layer_3.grad_weights , Hidden_Layer_3.grad_biases ## Same output as the derivations by hand.

(array([[64.],
        [64.]]),
 array([[16.]]))

Let's print the derivatives w.r.t. weights and biases  for $$H^{(2)}$$

In [27]:
Hidden_Layer_2.grad_weights ## Same output as the derivations by hand.

array([[32., 32.],
       [32., 32.]])

In [28]:
Hidden_Layer_2.grad_biases ### Same output as the derivations by hand.

array([[16., 16.]])

Let's print the derivatives w.r.t. weights and biases  for $$H^{(1)}$$

In [29]:
Hidden_Layer_1.grad_weights

matrix([[ 32.,  32.],
        [ 64.,  64.],
        [-32., -32.]])

In [30]:
Hidden_Layer_1.grad_biases

array([[32., 32.]])

Exactly the same values as derived by hand.
$$\tag*{$\blacksquare$}$$

In [31]:
## attempting batch normalization.


class Batch_Normalization_Layer:
    def forward_pass(self,x):
        self.x=x
        self.beta=np.random.standard_normal()
        self.gamma=np.random.standard_normal()
        ### adding epsilon so we dont accidentally divide by zero.
        self.epsilon=0.001
        ## gamma and beta used to scale and shift. 
        ## if gamma is learned to be the std and beta to be the mean, the batch norm layer will be
        ### the idendity activation!
        self.y=(x-np.mean(x))/np.sqrt((np.var(x)+self.epsilon))*self.gamma+self.beta
    def backward_pass(self,grad_vals):
        pass

### Trying out Gradient descent.
To make it easier, I'll use some simple data generation process.
The point is to implement gradient descent, in order to understand the workhorse behind Neural Networks.


Some notation:

Data Generation Process:
* Let $$x_1, x_2 , x_3, x_4$$ be vectors of length 10000 with values in [0,1] each.
* Let the noise $$\epsilon_i$$ be i.i.d Gaussians with $$\mu=0,\sigma^2=1$$
* Let the the real Y value be defined as $$Y_{real}=x_1-2x_2,+3x_3-4x_4+\epsilon$$

In [32]:
### Data Generation Process
np.random.seed(123)
x_1=np.random.uniform(low=0,high=1,size=10000)
x_2=np.random.uniform(low=0,high=1,size=10000)
x_3=np.random.uniform(low=0,high=1,size=10000)
x_4=np.random.uniform(low=0,high=1,size=10000)
epsilon=np.random.standard_normal(size=10000)
Y_real=x_1-2*x_2+3*x_3-4*x_4+epsilon

Let our prediction be $$Y_{pred}=\sum_{i=0}^nw_i*x_i$$ 
where $$x_0$$ is a dummy vector of 1's.
and $$w_i$$ are our weights

In [33]:
x_0=np.ones(10000)

In order to implement Gradient Descent, we want to minimize the cost function of the RSS (note we added a multiplication of 1/2 to make the derivative nicer)
$$J(w)=RSS=\frac12 \sum_{i=1}^n (Y_i-\hat{Y_i})^2$$


$$\frac{d}{dw}J(w)=\sum_{i=1}^n (Y_i-\hat{Y_i})x_i$$

Each column in X_matrix represents different x values.
a row in the X_matrix is one observation.

In [34]:
X_matrix=np.c_[x_0,x_1,x_2,x_3,x_4]
X_matrix

array([[1.        , 0.69646919, 0.66314239, 0.08647754, 0.89401513],
       [1.        , 0.28613933, 0.8397864 , 0.58416443, 0.48788606],
       [1.        , 0.22685145, 0.82365438, 0.47275191, 0.45711958],
       ...,
       [1.        , 0.98514494, 0.44507334, 0.12193146, 0.464396  ],
       [1.        , 0.22066212, 0.27707334, 0.84701671, 0.12011312],
       [1.        , 0.61329717, 0.27993699, 0.50973812, 0.34275199]])

grad_of_rss function will retrun the gradient at a given data point. will also work with a mini batch.

In [35]:
def grad_of_rss(x_vals,Y_Predictions,Y_Reals):
    delta=Y_Predictions-Y_Reals
    delta_weights=np.copy(x_vals)
    for i in range(len(delta)):
        delta_weights[i]=delta[i]*x_vals[i]
    #summing all columns into one cell and then taking the 
    #and finding the "true" values for the given input
    #note that we taking the avg to also prevent overflow/underflow.
    summations=delta_weights.sum(axis=0)/x_vals.shape[0]
    return summations

The Gradient Descent algorithm will get a step_size, the initial solution and the X values 
Along with the max_iter which is the maximum iterations of the algorithm.
it will also get a function that will return the gradient of the RSS at a given data point.
Weights will be initialized as 0.
it will return the weights vector.

The algorithm will run for max_iter iterations or until convergence.

In [36]:
def Gradient_Descent(X,Y,Gradient_calculator=grad_of_rss,step_size=0.01,max_iter=1000,
                     decay_rate=0,
                     mini_batch=-1,
                    gamma=0,v=0):
    W=np.zeros(5)
    X=np.copy(X_matrix)
    Y=np.copy(Y_real)
    #saving it just in case for exponential decay.
    step_size_original=step_size
    for i in range(max_iter):
        Y_preds=X @ W
        error=0.5*sum((Y_preds-Y_real)**2)
        #print('the RSS error in iteration',i,' is',error)
        #Update rule:
        '''
        if decay_rate has not been inputted, no exponential decay will happen,
        otherwise stepsize will be as follows:
        '''
        if decay_rate!=0:
            step_size=step_size_original*np.exp(-decay_rate * i)
            
        '''
        if mini_batch >0 then Stochastic Gradient Descent will happen.
        if mini_batch <=0 then Regular Gradient Descent will happen.
        '''
        if mini_batch>0:
            random_sample=random.sample(range(len(X)),mini_batch)
            # note that gradient calculator already does the averaging.
            grad_at_pts=(Gradient_calculator(x_vals=X[random_sample],Y_Predictions=Y_preds[random_sample],Y_Reals=Y[random_sample]))
            v=gamma*v+step_size*grad_at_pts
            W=W-v
        if mini_batch<=0:
            W=W-step_size*(Gradient_calculator(x_vals=X,Y_Predictions=Y_preds,Y_Reals=Y))
        ### Checking for convergence.
        Y_pred_new= X @ W
        new_error=0.5*sum((Y_pred_new-Y_real)**2)
        change_in_error=np.abs(new_error-error)
        if change_in_error<=0.1:
            print('Gradient Converged at iteration',i,' with an error value of ',error)
            return(W)
    print('Reached Maximum Iterations',max_iter)
    return W

Testing Gradient Descent:

In [37]:
final_weights=Gradient_Descent(X=X_matrix,Y=Y_real,Gradient_calculator=grad_of_rss,step_size=0.396,max_iter=1000,)
print("Gradient Descent Yields: W_1 =",round(final_weights[1],3),"|| W_2 =",round(final_weights[2],3),"|| W_3 =",round(final_weights[3],3),"|| W_4 =",round(final_weights[4],3))

Gradient Converged at iteration 141  with an error value of  4998.942604955297
Gradient Descent Yields: W_1 = 1.03 || W_2 = -1.942 || W_3 = 2.997 || W_4 = -3.906


We can see that Gradient Descent Yields pretty good Weights which are very close to the real weights of the Data Generation Process.

#### <span style="color:blue">Exponential Decay:</span>
Let's define exponential decay as:
$$\alpha_{epoch}=\alpha_{first} * e^ {-DecayRate*Epoch}$$
Where $$\alpha_{epoch}, \alpha_{first}$$ are the learning rate / step size.
by adding a decay_rate parameter to the gradient descent function we can easily implement exponential decay into our algorithm.
By setting decay_rate=0 as a default, we are back to having normal step_size calculations.

In [38]:
final_weights=Gradient_Descent(X=X_matrix,Y=Y_real,Gradient_calculator=grad_of_rss,step_size=0.396,max_iter=1000,decay_rate=0.1)
print("Exponential Decay Yields: W_1 =",round(final_weights[1],3),"|| W_2 =",round(final_weights[2],3),"|| W_3 =",round(final_weights[3],3),"|| W_4 =",round(final_weights[4],3))

Gradient Converged at iteration 83  with an error value of  10958.847879299516
Exponential Decay Yields: W_1 = 0.124 || W_2 = -0.726 || W_3 = 0.71 || W_4 = -1.303


By having the same step_size as before but this time declaring a decay rate=0.1, we can see that the algorithm converged faster, but the error is worse. (premature stopping)

#### <span style="color:purple">Stochastic Gradient Descent:</span>

The main insight from stochastic gradient descent is that the gradient is an expectation. The expected value can be approximated using a small batch size. More specifically, we can sample a batch_size uniformly WITH REPLACEMENT from the train set.
Let $$B^{sgd}$$ be the batch size and RSS the cost function.
we can estimate the gradient as
$$ \hat g(x)= \frac{1}{B^{sgd}} \frac{d}{dw}\sum_{i=1}^{B^{sgd}} L(x^{i},y^{i},w) $$

adding mini_batch to the Gradient Descent function. By setting it to a positive integer, the algorithm will perform stochastic gradient descent, with the batch size which was inputted. By default, batch_size=-1 which will tell the algorithm to perform the regular Gradient Descent.

Let's set mini_batch=64 and see what happens.

In [39]:
final_weights=Gradient_Descent(X=X_matrix,Y=Y_real,Gradient_calculator=grad_of_rss,step_size=0.396,max_iter=20000,mini_batch=64)
print("Stochastic Gradient Descent Yields: W_1 =",round(final_weights[1],3),"|| W_2 =",round(final_weights[2],3),"|| W_3 =",round(final_weights[3],3),"|| W_4 =",round(final_weights[4],3))

Gradient Converged at iteration 152  with an error value of  5001.882569035912
Stochastic Gradient Descent Yields: W_1 = 1.015 || W_2 = -1.992 || W_3 = 3.014 || W_4 = -3.916


The algorithm converged after more iterations(when compared to vanilla GD), but these iterations were faster(in real life time) than the normal GD algorithm. The error value is similar.

#### <span style="color:green">Momentum</span>

The velocity v accumulates the gradient elements. also the larger the gamma is relative to the step size, the more previous gradients affect the current direction of the momentum. 

In order to add the ability for momentum in the Gradient Descent function
I will add two parameters
v for initial velocity and gamma for the momentum parameter.
both will be set to 0 by default. if both of them are non zero(along with mini_batch) the algorithm will perform Stochastic Gradient Descent with Momentum.
Running Stochastic Gradient Descent with Momentum:

In [40]:
final_weights=Gradient_Descent(X=X_matrix,Y=Y_real,Gradient_calculator=grad_of_rss,step_size=0.396,max_iter=20000,mini_batch=64,gamma=0.222,v=1)
print("Momentum Gradient Descent Yields: W_1 =",round(final_weights[1],3),"|| W_2 =",round(final_weights[2],3),"|| W_3 =",round(final_weights[3],3),"|| W_4 =",round(final_weights[4],3))

Gradient Converged at iteration 442  with an error value of  5000.146305334952
Momentum Gradient Descent Yields: W_1 = 1.002 || W_2 = -2.029 || W_3 = 2.989 || W_4 = -3.967


$$\tag*{$\blacksquare$}$$